In [9]:
!pip install einops -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.1 MB/s eta 0:00:00


In [2]:
!pip install accelerate transformers datasets peft bitsandbytes --upgrade

  Using cached accelerate-0.25.0-py3-none-any.whl (265 kB)
  Using cached transformers-4.36.1-py3-none-any.whl (8.3 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [5]:
from peft import PeftModel
from transformers import AutoModelForCausalLM,AutoTokenizer,TrainingArguments,Trainer,DataCollatorForLanguageModeling, BitsAndBytesConfig
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, get_peft_model

In [7]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1_5",trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [10]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-1_5",
    device_map={"":0},
    trust_remote_code=True,
    quantization_config = bnb_config
)
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=['Wqkv','out_proj'],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model,lora_config)
model.print_trainable_parameters()

pytorch_model.bin:   0%|          | 0.00/2.84G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/69.0 [00:00<?, ?B/s]

trainable params: 4,718,592 || all params: 1,422,989,312 || trainable%: 0.3315971497613047


In [14]:
def tokenize(sample):
  tok = tokenizer(sample['text'],padding=True,truncation=True,max_length=512)
  return tok

dataset = load_dataset("BI55/MedText",split="train")
data_df = dataset.to_pandas()
data_df


,Prompt,Completion
0,A 50-year-old male presents with a history of ...,This patient's history of recurrent kidney sto...
1,"A 7-year-old boy presents with a fever, headac...","This child's symptoms of a red, bulging tympan..."
2,A 35-year-old woman presents with a persistent...,While the symptoms might initially suggest ast...
3,A 50-year-old male presents with severe abdomi...,The patient's symptoms suggest an incarcerated...
4,A newborn baby presents with eye redness and a...,The infant's symptoms suggest neonatal conjunc...
...,...,...
1407,A 55-year-old male with a history of chronic o...,While this patient's symptoms could be due to ...
1408,Can diet and lifestyle changes help manage vit...,While there is no specific diet or lifestyle m...
1409,A 50-year-old female presents with right shoul...,This patient's shoulder and arm pain following...
1410,A 60-year-old female with high cholesterol lev...,In addition to a diet low in saturated fats an...


In [27]:
data_df["text"] = data_df[["Prompt", "Completion"]].apply(lambda x: "Prompt: " + x["Prompt"] + " Completion: " + x["Completion"], axis=1)
data = Dataset.from_pandas(data_df)

tok_ds = data.map(tokenize,desc="Tokenizing data",remove_columns=data.column_names)
tok_ds

Tokenizing data:   0%|          | 0/1412 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 1412
})

In [31]:
training_arguments = TrainingArguments(
    output_dir='.',
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    save_strategy="epoch",
    logging_steps=500,
    num_train_epochs=5
)

trainer = Trainer(
    model=model,
    train_dataset=tok_ds,
    args=training_arguments,
    data_collator=DataCollatorForLanguageModeling(tokenizer,mlm=False)
)

In [32]:
trainer.train()

Step,Training Loss
500,1.302000


Checkpoint destination directory ./checkpoint-176 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=880, training_loss=1.2098678935657847, metrics={'train_runtime': 783.9049, 'train_samples_per_second': 9.006, 'train_steps_per_second': 1.123, 'total_flos': 9298934155689984.0, 'train_loss': 1.2098678935657847, 'epoch': 4.99})